## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-11-02-24-40 +0000,nyt,Trump Pardons the Husband of a Republican Cong...,https://www.nytimes.com/2025/11/10/us/politics...
1,2025-11-11-02-22-00 +0000,wsj,TPG to Seek Bids for Southeast Asia School Ope...,https://www.wsj.com/business/deals/tpg-to-seek...
2,2025-11-11-02-17-39 +0000,nyt,Supreme Court to Hear Major Challenge to Mail-...,https://www.nytimes.com/2025/11/10/us/politics...
3,2025-11-11-02-12-33 +0000,nyt,F.B.I. Director Is Said to Have Made a Pledge ...,https://www.nytimes.com/2025/11/10/world/europ...
4,2025-11-11-02-12-03 +0000,nyt,Trump Threatens to Sue the BBC for $1 Billion ...,https://www.nytimes.com/2025/11/10/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,43
61,shutdown,38
60,government,21
59,end,17
107,deal,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
24,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...,145
125,2025-11-10-21-11-40 +0000,nypost,Trump pledges to ‘abide’ by deal to end govern...,https://nypost.com/2025/11/10/us-news/trump-pl...,141
7,2025-11-11-02-00-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...,141
78,2025-11-10-22-53-00 +0000,wsj,The decision by eight members of the Senate De...,https://www.wsj.com/politics/policy/democrats-...,131
251,2025-11-10-13-44-58 +0000,nypost,"Democrats melt down, call for Chuck Schumer’s ...",https://nypost.com/2025/11/10/us-news/democrat...,123


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
24,145,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...
7,82,2025-11-11-02-00-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...
236,69,2025-11-10-14-58-51 +0000,nypost,Supreme Court rejects Kim Davis bid to overtur...,https://nypost.com/2025/11/10/us-news/supreme-...
248,51,2025-11-10-14-01-26 +0000,nypost,Ex-French President Nicolas Sarkozy to be rele...,https://nypost.com/2025/11/10/world-news/franc...
234,49,2025-11-10-15-03-47 +0000,cbc,8 people dead in car explosion near Red Fort i...,https://www.cbc.ca/news/world/india-explosion-...
211,42,2025-11-10-16-35-00 +0000,wsj,Sen. Bernie Sanders and a group of Democratic ...,https://www.wsj.com/politics/policy/the-politi...
43,42,2025-11-11-00-04-13 +0000,nypost,"Major break in Long Island cold case murder, r...",https://nypost.com/2025/11/10/us-news/major-br...
274,36,2025-11-10-11-40-11 +0000,nypost,Trump threatens BBC with $1B lawsuit over doct...,https://nypost.com/2025/11/10/us-news/trump-la...
261,36,2025-11-10-12-30-00 +0000,wsj,Private-Label Food Maker TreeHouse to Go Priva...,https://www.wsj.com/business/retail/private-la...
223,36,2025-11-10-15-58-48 +0000,nypost,NYC realtor fired over controversial Zohran Ma...,https://nypost.com/2025/11/10/business/nyc-rea...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
